In [14]:
from metadata_utils import get_meta
from llm_utils import ask_llm
from transcript_utils import srt_to_text
from chunking_utils import chunker, token_counter
from chonkie import Chunk

In [40]:
from pathlib import Path

files = [file for file in Path("../files/rotl").iterdir() if ".srt" in file.name]

results_dir = Path("summaries_test")
results_dir.mkdir(exist_ok=True)

chunks_dir = Path("chunks")
chunks_dir.mkdir(exist_ok=True)


def who_is(subject, exclude=[]):
    results = []
    for file in files:
        file_name, episode_number, episode_date, episode_title = get_meta(file)
        text = srt_to_text(file)
        if subject in text and all(name not in text for name in exclude):
            chunks = chunker(text)
            for chunk in chunks:
                if isinstance(chunk, Chunk) and chunk.text.count(subject):
                    results.append(chunk.text)
    chunks_path = chunks_dir / f"{subject}_chunks.txt"
    with open(chunks_path, "w") as f:
        f.write("\n".join(results))
    question = f"Who is {subject}?"
    context = f"{"\n".join(results)}\n\n\n{question}"
    raw_answer = ask_llm(context)
    answer = raw_answer.strip().replace(". ", ".\n")
    out_path = results_dir / f"{subject}.txt"

    with open(out_path, "w") as f:
        f.write(answer)

    return answer

In [41]:
exclude = ["Eleanor Rigby", "Eleanor Roosevelt", "Eleanor Braun"]
answer = who_is("Eleanor", exclude=exclude)

print(answer)

Eleanor is John Roderick's daughter.
She is a young child who often accompanies John on his travels.
She is known for her love of Iron Man comic books, Goldfish crackers, and her unique way of counting numbers, which includes the number "one hundred and thirty and two." Eleanor is also known for her stubbornness and her tendency to jump on things, such as John's air mattress.
Despite her young age, she has a strong personality and can be quite opinionated.
John often jokes about her and refers to her as his "mini-me."
